In [1]:
import numpy as np

from functions import *

In [2]:
a = np.array([1, 2, 3, 4, 5, 6, 7, 8 ])
print(a)
print(np.clip(a, a_min = 5, a_max= np.inf))

[1 2 3 4 5 6 7 8]
[5. 5. 5. 5. 5. 6. 7. 8.]


In [2]:
nx = 100
ny = 100
num_species = 3
nB = 15

f = np.random.random((num_species, 9, nx, ny))

phi = np.ones(num_species)
molecular_weight = np.ones(num_species)

In [3]:
def bgk_step(f, molecular_weight, phi, nB):

    #################### Before streaming ####################

    rho_s, ux_s, uy_s, rho_mix, p_mix = calculate_moment(f, phi)

    m_mix = calculate_m_mix(rho_s, rho_mix, molecular_weight)

    CHI_sc = calculate_CHI(m_mix, molecular_weight, nB)

    lambda_s = calculate_lambda(rho_mix, p_mix, molecular_weight, nB)

    ux_star_s, uy_star_s = calculate_u_star(CHI_sc, rho_s, rho_mix, ux_s, uy_s)

    feq = equilibrium(f, rho_s, phi, ux_star_s, uy_star_s)

    g_dagger_s =calculate_g_dagger(f, feq, lambda_s)

    #################### After streaming ####################

    f_streamed = lattice_stream(f)

    rho_s, ux_s, uy_s, rho_mix, p_mix = calculate_moment(f_streamed, phi)

    m_mix = calculate_m_mix(rho_s, rho_mix, molecular_weight)

    lambda_s = calculate_lambda(rho_mix, p_mix, molecular_weight, nB)

    CHI_sc = calculate_CHI(m_mix, molecular_weight, nB)

    Chi_S = post_stream_Chi_S(CHI_sc, rho_s, rho_mix)

    ux_dagger, uy_dagger, jx_s, jy_s = solve_ms_fluxes(lambda_s, Chi_S, CHI_sc, rho_s, rho_mix, ux_s, uy_s, theta=theta)

    ux_star_dagger_s, uy_star_dagger_s = calculate_u_star(CHI_sc, rho_s, rho_mix, ux_dagger, uy_dagger)

    feq_dagger = equilibrium(f_streamed, rho_s, phi, ux_star_dagger_s, uy_star_dagger_s)

    f_new = distribution_semi_implicit(feq_dagger, g_dagger_s, lambda_s)

    return f_new

In [4]:
bgk_step(f, molecular_weight, phi, nB)

array([[[[[[ 5.71658726e+00,  5.89632307e+00,  3.29597677e+00, ...,
             4.01488694e+00,  4.98799304e+00,  4.72758417e+00],
           [ 6.16667304e+00,  4.05866673e+00,  4.77919151e+00, ...,
             3.72770185e+00,  3.45789679e+00,  4.85986763e+00],
           [ 3.91321721e+00,  4.58075286e+00,  3.84466415e+00, ...,
             3.96616181e+00,  4.65455046e+00,  4.15448620e+00],
           ...,
           [ 6.30481227e+00,  4.26236464e+00,  5.21832525e+00, ...,
             4.98377444e+00,  3.42448896e+00,  4.88404844e+00],
           [ 4.65174727e+00,  4.34341307e+00,  5.23005384e+00, ...,
             4.56418787e+00,  5.17686712e+00,  5.58848117e+00],
           [ 5.22704684e+00,  4.56945043e+00,  5.58630492e+00, ...,
             4.92212369e+00,  5.62520501e+00,  4.57676917e+00]],

          [[ 8.85137347e-02,  1.19491460e+00,  1.97617406e-01, ...,
             6.71922702e-01,  3.58225715e-01,  1.05720660e+00],
           [ 1.01870591e-01,  9.55422667e-01,  7.04713186e

In [10]:
b = np.array([
    [1, 2, 3, 4, 5, 6, 7, 8],
    [1, 2, 3, 4, 5, 6, 7, 8],
    [1, 2, 3, 4, 5, 6, 7, 8],
    ]
)
b[1,:] *= 2
b[2,:] *= 3
print(b)

[[ 1  2  3  4  5  6  7  8]
 [ 2  4  6  8 10 12 14 16]
 [ 3  6  9 12 15 18 21 24]]


In [16]:
delta = a[None, :] -b
print(delta)

[[  0   0   0   0   0   0   0   0]
 [ -1  -2  -3  -4  -5  -6  -7  -8]
 [ -2  -4  -6  -8 -10 -12 -14 -16]]
